In this notebook, we will show using [PyNN](http://neuralensemble.org/docs/PyNN/index.html) how a neuron may may learn a tuned receptive field.

We will use the STDP mechanism @ http://neuralensemble.org/docs/PyNN/0.7/dynamicsynapses.html

In [ ]:
import numpy as np

In [ ]:
import pyNN.nest as sim
from pyNN.random import RandomDistribution, NumpyRNG

# defining a feed-forward SNN with STDP


In [ ]:
# === Define parameters ========================================================
dt         = 0.5           # (ms)
simtime    = 3000.0        # (ms)
simtime    = 20000.0        # (ms)

N_inputs = 60
#N_inputs = 360

lambda_n = .5 # (Hz) rate of poisson noise
theta = .50
A = 6.
B = .61803

In [ ]:
def tuning_function(N_inputs, lambda_n=lambda_n, A=A, theta=theta, B=B):
    return lambda_n + A*np.exp((np.cos(2*np.pi*(np.linspace(0, 1, N_inputs)-theta))-1)/B**2)
rate = tuning_function(N_inputs, lambda_n=lambda_n, A=A, theta=theta, B=B)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
fig_width = 15
phi = (np.sqrt(5)+1)/2

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(rate)
ax.set_ylim(0)
ax.set_xlabel('input')
ax.set_ylabel('rate (Hz)');

In [ ]:
weight0 = .15
weights = weight0 * np.random.rand(N_inputs, N_inputs)
#weights[weights<weight0/1.1] = 0

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width))
ax.imshow(weights.T)
ax.set_xlabel("Ouput index")
ax.set_ylabel('Input index');

In [ ]:
# === Build the network ========================================================
sim.setup(timestep=dt)


spike_input = sim.Population(N_inputs, sim.SpikeSourcePoisson(rate=rate, duration=simtime), label="expoisson")        
spike_input.record('spikes')

thalamocort_cell = sim.IF_cond_alpha()

tc_cells = sim.Population(N_inputs, thalamocort_cell)
tc_cells.record('spikes')

#Synapses type
eta = .01
A_plus, A_minus = 1.*eta, 1.2*eta

causal_hebbian_rule = sim.SpikePairRule(tau_plus=30., tau_minus=40.,
                                      A_plus=A_plus, A_minus=A_minus)

causal_STDP_syn = sim.STDPMechanism(
    timing_dependence=causal_hebbian_rule,
    #weight_dependence=sim.AdditiveWeightDependence(w_min=0, w_max=1.),
    weight_dependence=sim.MultiplicativeWeightDependence(w_min=0, w_max=.5),
    delay=1.5,)

#connectivity using the projection method
connections = sim.Projection(spike_input, tc_cells,
                                    #connector=sim.AllToAllConnector(),# 
                                    connector=sim.FixedProbabilityConnector(.9),
                                    synapse_type=causal_STDP_syn,
                                    receptor_type='excitatory',
                                    label="excitatory connections")

connections.set(weight=weights)

# === Run simulation ===========================================================
sim.run(simtime)
sim.end()

In [ ]:
from tools import plot_spiketrains
fig, axs = plt.subplots(2, 1, figsize=(fig_width, fig_width/phi))
fig, axs[0] = plot_spiketrains(spike_input, color='g', simtime=simtime, fig=fig, ax=axs[0])
fig, axs[1] = plot_spiketrains(tc_cells, color='r', simtime=simtime, fig=fig, ax=axs[1])

In [ ]:
from tools import histogram
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
fig, ax = histogram(spike_input, color='g', simtime=simtime, fig=fig, ax=ax)
fig, ax = histogram(tc_cells, color='r', simtime=simtime, fig=fig, ax=ax)

In [ ]:
weights = connections.get('weight', format="array")


fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width))
ax.imshow(weights.T)
ax.set_xlabel("Output index")
ax.set_ylabel('Input index');